In [1]:
import random

In [2]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [77]:
print(problem(5,42))

[[0], [1], [0], [4], [0], [1], [4], [4], [4], [1, 3], [0, 1], [2], [1], [0], [0, 2], [2, 4], [3], [3], [4], [2, 4], [0], [1], [0, 1], [3], [2, 3]]


In [3]:
import numpy as np

In [37]:
from gx_utils import *

In [38]:
import logging


def greedy(N):
    goal = set(range(N))
    covered = set()
    solution = list()
    all_lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    while goal != covered:
        x = all_lists.pop(0)
        if not set(x) < covered:
            solution.append(x)
            covered |= set(x)

    logging.info(
        f"Greedy solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )
    logging.debug(f"{solution}")

In [39]:
logging.getLogger().setLevel(logging.INFO)
for N in [5, 10, 20, 100, 500, 1000]:
    greedy(N)

INFO:root:Greedy solution for N=5: w=5 (bloat=0%)
INFO:root:Greedy solution for N=10: w=13 (bloat=30%)
INFO:root:Greedy solution for N=20: w=46 (bloat=130%)
INFO:root:Greedy solution for N=100: w=332 (bloat=232%)
INFO:root:Greedy solution for N=500: w=2162 (bloat=332%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)


In [40]:
import logging
from random import seed, choice
from typing import Callable

logging.basicConfig(format="%(message)s", level=logging.INFO)

In [41]:
class State:
    def __init__(self, data: np.ndarray):
        self._data = data.copy()
        self._data.flags.writeable = False

    def __hash__(self):
        return hash(bytes(self._data))

    def __eq__(self, other):
        return bytes(self._data) == bytes(other._data)

    def __lt__(self, other):
        return bytes(self._data) < bytes(other._data)

    def __str__(self):
        return str(self._data)

    def __repr__(self):
        return repr(self._data)

    @property
    def data(self):
        return self._data

    def copy_data(self):
        return self._data.copy()

In [42]:
def search(
    initial_state: State,
    goal_test: Callable,
    parent_state: dict,
    state_cost: dict,
    priority_function: Callable,
    unit_cost: Callable,
    input_state: State

):
'''
frontier = PriorityQueue()
parent_state.clear()
state_cost.clear()

# initial_state is [0]
state = initial_state
parent_state[0] = None
#state_cost[state] is set to 1
state_cost[0] = len(state)

while state is not None and not goal_test(state):
    for elem in input_state:
        if elem == state:
            continue
        cost = len(elem)
        
        index = input_state.index(elem)
        new_state = State(np.array(elem))

        intersection = list(set(state) & set(elem))
        # check if new node contains element different from ones in the state
        if len(intersection) !== 0:
            parent_state[index] = cost
            frontier.push(new_state, p=priority_function(new_state))
        else:
            continue
'''

frontier = PriorityQueue()
parent_state.clear()
state_cost.clear()

state = initial_state
parent_state[state] = None
state_cost[state] = 0

while state is not None and not goal_test(state):
    for a in possible_actions(state):
        new_state = result(state, a)
        cost = unit_cost(a)
        if new_state not in state_cost and new_state not in frontier:
            parent_state[new_state] = state
            state_cost[new_state] = state_cost[state] + cost
            frontier.push(new_state, p=priority_function(new_state))
            logging.debug(f"Added new node to frontier (cost={state_cost[new_state]})")
        elif new_state in frontier and state_cost[new_state] > state_cost[state] + cost:
            old_cost = state_cost[new_state]
            parent_state[new_state] = state
            state_cost[new_state] = state_cost[state] + cost
            logging.debug(f"Updated node cost in frontier: {old_cost} -> {state_cost[new_state]}")
    if frontier:
        state = frontier.pop()
    else:
        state = None

path = list()
s = state
while s:
    path.append(s.copy_data())
    s = parent_state[s]

logging.info(f"Found a solution in {len(path):,} steps; visited {len(state_cost):,} states")
return list(reversed(path))

'''
    frontier = PriorityQueue()
    parent_state.clear()
    state_cost.clear()

    state = initial_state
    parent_state[state] = None
    state_cost[state] = 0

    while state is not None and not goal_test(state):
        for a in possible_actions(state):
            new_state = result(state, a)
            cost = unit_cost(a)
            if new_state not in state_cost and new_state not in frontier:
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                frontier.push(new_state, p=priority_function(new_state))
                logging.debug(f"Added new node to frontier (cost={state_cost[new_state]})")
            elif new_state in frontier and state_cost[new_state] > state_cost[state] + cost:
                old_cost = state_cost[new_state]
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                logging.debug(f"Updated node cost in frontier: {old_cost} -> {state_cost[new_state]}")
        if frontier:
            state = frontier.pop()
        else:
            state = None

    path = list()
    s = state
    while s:
        path.append(s.copy_data())
        s = parent_state[s]

    logging.info(f"Found a solution in {len(path):,} steps; visited {len(state_cost):,} states")
    return list(reversed(path))
    '''

In [68]:
input_state = sorted(problem(5, seed=42), key=lambda l: len(l))
initial_state = State(np.array(problem(5,42)[0]))
print(input_state)

[[0], [1], [0], [4], [0], [1], [4], [4], [4], [2], [1], [0], [3], [3], [4], [0], [1], [3], [1, 3], [0, 1], [0, 2], [2, 4], [2, 4], [0, 1], [2, 3]]


In [44]:
GOAL = State(np.array(range(5)))
logging.info(f"Goal:\n{GOAL}")
print(GOAL)


def goal_test(state):
    return state == GOAL

INFO:root:Goal:
[0 1 2 3 4]


[0 1 2 3 4]


# Breadth-first

In [45]:
parent_state = dict()
state_cost = dict()

final = search(
    initial_state=initial_state,
    goal_test=goal_test,
    parent_state=parent_state,
    state_cost=state_cost,
    priority_function=lambda s: len(state_cost),
    unit_cost=lambda a: 1,
)

TypeError: unhashable type: 'list'

In [82]:
l = [[1,0], [3, 4, 0], [0,1]]

for e in l:
    print(l.index(e))

0
1
2
